#### 1. Haced un programa que resuelva la colisión de _N_ partículas de masa _m_ y radio _R_ (problema en 2 dimensiones).

#### 2. Considerar tanto condiciones de contorno periódicas, como fijas (paredes rígidas).

Primero consideramos una caja 2x2 con condiciones de contorno periódicas:

In [ ]:
import numpy as np

# Definimos los parámetros del sistema:
N=50
Nt=1000                               # Aumentar para alargar la simulación
m=1
rscale=5.e6
L=1
R=2e-10*rscale                        # Radio atómico
R=0.01
tscale=1e9                            # en nanosegundos
sbar = 353*rscale/tscale              # velocidad promedio del Ar a 300K  DEMASIADO RAPIDO
sbar= 0.01
FPS = 30
dt= 1./FPS


t=np.linspace(0,10,Nt)

yy0=2*np.random.random([N,1])-1
yy1=np.zeros([N,Nt-1])
yy=np.concatenate((yy0,yy1),axis=1)

xx0=2*np.random.random([N,1])-1
xx1=np.zeros([N,Nt-1])
xx=np.concatenate((xx0,xx1),axis=1)

vx0=np.sqrt(2)*sbar*np.random.random([N,1])-np.sqrt(2)*sbar/2
vx1=np.zeros([N,Nt-1])
vx=np.concatenate((vx0,vx1),axis=1)

vy0=np.sqrt(2)*sbar*np.random.random([N,1])-np.sqrt(2)*sbar/2
vy1=np.zeros([N,Nt-1])
vy=np.concatenate((vy0,vy1),axis=1)



for n in range(1,Nt):
    for i in range(N):
        vx[i,n]=vx[i,n-1]  
        vy[i,n]=vy[i,n-1]
        xx[i,n]=xx[i,n-1]+vx[i,n-1]*t[n]
        yy[i,n]=yy[i,n-1]+vy[i,n-1]*t[n]
        
        #introducimos las condiciones de contorno PERIÓDICAS
        if abs(xx[i,n])>=L:
            xx[i,n]=-xx[i,n-1]
        if abs(yy[i,n])>=L:
            yy[i,n]=-yy[i,n-1]    #al chocar con las paredes, se aceleran!!
            
        #ahora introducimos las colisiones:
        for j in range(i,N):
            if (np.sqrt((xx[i,n]-xx[j,n])**2 +(yy[i,n]-yy[j,n])**2)<=R) & (i!=j): #quizas R demasiado pequeño?
                Vcmx=0.5*(vx[i,n]+vx[j,n])
                Vcmy=0.5*(vy[i,n]+vy[j,n])
                modsep=np.sqrt((xx[i,n]-xx[j,n])**2+(yy[i,n]-yy[j,n])**2)
                modv=np.sqrt((vx[i,n])**2+(vy[i,n])**2)
                
                vx[i,n]=vx[i,n]-Vcmx-2*modv*(xx[i,n]-xx[j,n])/modsep
                vy[i,n]=vy[i,n]-Vcmy-2*modv*(yy[i,n]-yy[j,n])/modsep
                vx[j,n]=-vx[i,n]
                vy[j,n]=-vy[i,n]



import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
dimfig=6

#Creamos figura y plot
fig,ax= plt.subplots()

#dimensiones de la figura
fig.set_size_inches(dimfig,dimfig)

#Primeros datos representados
sp,= ax.plot(xx[:,0],yy[:,0],ls='',marker='.',markersize=3)
ax.set_aspect('equal')

def animate(i):
    sp.set_data(xx[:,i],yy[:,i]) #Actualizamos datos
    plt.xlim(-L,L)
    plt.ylim(-L,L)

#Hacemos la animación
ani=FuncAnimation(fig,animate,frames=len(t), interval=10)
HTML(ani.to_jshtml())

Ahora consideramos la condiciones de contorno fijas:

In [ ]:
import numpy as np

# Definimos los parámetros del sistema:
N=5
Nt=1000                               # Aumentar para alargar la simulación
m=1
L=.5
rscale=5.e6
R=0.01                        # Radio atómico
tscale=1e9                            # en nanosegundos
sbar = 353*rscale/tscale              # velocidad promedio del Ar a 300K
FPS = 30
dt= 1./FPS


t=np.linspace(0,10,Nt)

yy0=2*L*np.random.random([N,1])-L
yy1=np.zeros([N,Nt-1])
yy=np.concatenate((yy0,yy1),axis=1)

xx0=2*L*np.random.random([N,1])-L
xx1=np.zeros([N,Nt-1])
xx=np.concatenate((xx0,xx1),axis=1)

vx0=.005*np.random.random([N,1])-.0025
vx1=np.zeros([N,Nt-1])
vx=np.concatenate((vx0,vx1),axis=1)

vy0=.005*np.random.random([N,1])-.0025
vy1=np.zeros([N,Nt-1])
vy=np.concatenate((vy0,vy1),axis=1)


it=0
for n in range(1,Nt):
    for i in range(N):
        vx[i,n]=vx[i,n-1]  
        vy[i,n]=vy[i,n-1]
        xx[i,n]=xx[i,n-1]+vx[i,n-1]*t[n]
        yy[i,n]=yy[i,n-1]+vy[i,n-1]*t[n]
        
        #introducimos las condiciones de contorno FIJAS
        if abs(xx[i,n])>=L-R:
            vx[i,n]=-vx[i,n-1]
           
        if abs(yy[i,n])>=L-R:
            vy[i,n]=-vy[i,n-1]    #al chocar con las paredes, se aceleran!!
            
        #ahora introducimos las colisiones:
        for j in range(i,N):
            if (np.sqrt((xx[i,n]-xx[j,n])**2 +(yy[i,n]-yy[j,n])**2)<=2*R) & (i!=j):
                Vcmx=0.5*(vx[i,n]+vx[j,n])
                Vcmy=0.5*(vy[i,n]+vy[j,n])
                modsep=np.sqrt((xx[i,n]-xx[j,n])**2+(yy[i,n]-yy[j,n])**2)
                modv=np.sqrt((vx[i,n])**2+(vy[i,n])**2)
                
                vx[i,n]=vx[i,n]-Vcmx-2*modv*(xx[i,n]-xx[j,n])/modsep
                vy[i,n]=vy[i,n]-Vcmy-2*modv*(yy[i,n]-yy[j,n])/modsep
                vx[j,n]=-vx[i,n]
                vy[j,n]=-vy[i,n]
                it+=1
                #print(modsep,modv,sep='---')
print(it)

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
dimfig=6

#Creamos figura y plot
fig,ax= plt.subplots()

#dimensiones de la figura
fig.set_size_inches(dimfig,dimfig)

#Primeros datos representados
sp,= ax.plot(xx[:,0],yy[:,0],ls='',marker='.',markersize=3)
ax.set_aspect('equal')

def animate(i):
    sp.set_data(xx[:,i],yy[:,i]) #Actualizamos datos
    plt.xlim(-L,L)
    plt.ylim(-L,L)

#Hacemos la animación
ani=FuncAnimation(fig,animate,frames=len(t), interval=10)
HTML(ani.to_jshtml())

#### 3. Considerando paredes rígidas, obtened el histograma del módulo de la velocidad y comprobad que la distribución de velocidades sigue la distribución de Maxwell-Boltzmann en 2 dimensiones

$$\rho (v)=\frac{mv}{k_B T}e^{-\frac{1}{2}\frac{mv^2}{k_B T}}$$

#### Ayuda: usar el teorema de equipartición para estimar el valor de $k_BT$.

La expresión más general del teorema de equipartición es 
$$\langle x_m \frac{\partial H}{\partial x_n} \rangle =k_B T\delta_{m,n}.$$

Si tomamos $x$ como una componente del momento lineal de una partícula ($p_i$), y usando las ecuaciones canónicas de Hamilton, otenemos $$\langle x_m \frac{\partial H}{\partial x_n} \rangle= \langle p_k\frac{dq_k}{dt}\rangle=\langle p_i v_i\rangle=\langle m_i v_i^2\rangle.$$

Ahora consideramos $N$ partículas y $d$ dimensiones, y llegamos a la expresión $$ k_BT=\frac{1}{Nd}\langle \sum_{i=1}^N m_iv_i^2\rangle.$$

Dividiendo ambos lados entre dos, y con $d=2$, llegamos a una expresión más interpretable y fácil de calcular: $$k_bT=\frac{1}{N}\langle T\rangle\ ,$$
donde $\langle T\rangle$ es la energía cinética promedio de las N partículas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Definimos los parámetros del sistema:
N=264
Nt=1000                               # Aumentar para alargar la simulación
m=1
L=1
rscale=5.e6
R=2e-10*rscale                        # Radio atómica
tscale=1e9                            # en nanosegundos
sbar = 353*rscale/tscale              # velocidad promedio del Ar a 300K
FPS = 30
dt= 1./FPS


t=np.linspace(0,10,Nt)

yy0=2*np.random.random([N,1])-1
yy1=np.zeros([N,Nt-1])
yy=np.concatenate((yy0,yy1),axis=1)

xx0=2*np.random.random([N,1])-1
xx1=np.zeros([N,Nt-1])
xx=np.concatenate((xx0,xx1),axis=1)

vx0=.005*np.random.random([N,1])-.0025
vx1=np.zeros([N,Nt-1])
vx=np.concatenate((vx0,vx1),axis=1)

vy0=.005*np.random.random([N,1])-.0025
vy1=np.zeros([N,Nt-1])
vy=np.concatenate((vy0,vy1),axis=1)


s=np.linspace(1,N,N)
for n in range(1,Nt):
    for i in range(N):
        vx[i,n]=vx[i,n-1]  
        vy[i,n]=vy[i,n-1]
        xx[i,n]=xx[i,n-1]+vx[i,n-1]*t[n]
        yy[i,n]=yy[i,n-1]+vy[i,n-1]*t[n]

        #introducimos las condiciones de contorno FIJAS
        if abs(xx[i,n])>=k:
            vx[i,n]=-vx[i,n-1]
        if abs(yy[i,n])>=k:
            vy[i,n]=-vy[i,n-1]    #al chocar con las paredes, se aceleran!!        
        #ahora introducimos las colisiones:
        for j in range(i,N):
            if (np.sqrt((xx[i,n]-xx[j,n])**2 +(yy[i,n]-yy[j,n])**2)<=R) & (i!=j):
                Vcmx=0.5*(vx[i,n]+vx[j,n])
                Vcmy=0.5*(vy[i,n]+vy[j,n])
                modsep=np.sqrt((xx[i,n]-xx[j,n])**2+(yy[i,n]-yy[j,n])**2)
                modv=np.sqrt((vx[i,n])**2+(vy[i,n])**2)

                vx[i,n]=vx[i,n]-Vcmx-2*modv*(xx[i,n]-xx[j,n])/modsep
                vy[i,n]=vy[i,n]-Vcmy-2*modv*(yy[i,n]-yy[j,n])/modsep
                vx[j,n]=-vx[i,n]
                vy[j,n]=-vy[i,n]


                
modvels=np.sqrt(vx**2+vy**2)
kT=1/(2*N)*sum(vx**2+vy**2)[1]
print('kT =',kT)
plt.hist(modvels, density=True )
plt.xlabel('Velocidades')
plt.ylabel('Número de partículas')
plt.title('Histograma de Velocidades')
plt.show()

rho=m*modvels/kT *np.exp(-0.5*m*(modvels)**2/kT)
plt.scatter(modvels,rho/np.max(rho),marker='.')
plt.xlabel('Velocidades')
plt.ylabel('Densidad de partículas')
plt.title('Distribución de Maxwell-Boltzmann')
plt.show()

#### 4. Para las condiciones de contorno fijas, obtenga el promedio de colisiones con una de las paredes por unidad de tiempo. Variad el tamaño de la caja y comprobad como varía el número de colisiones. ¿Qué podéis deducir de esta relación?

Para tener mayor precisión estadística mediremos las colisiones en todas las paredes, y dividiremos entre cuatro para obtener las colisiones en una sola cara, que llamaremos $\sigma$.
Al dividir entre el tiempo, en nuestro caso $t=10s$, obtendremos las colisiones por segundo: $\sigma /t = 0.1\sigma$.

In [ ]:
import numpy as np

# Definimos los parámetros del sistema:
N=200
Nt=1000                               # Aumentar para alargar la simulación
m=1
L=10
rscale=5.e6
R=2e-10*rscale                        # Radio atómica
tscale=1e9                            # en nanosegundos
sbar = 353*rscale/tscale              # velocidad promedio del Ar a 300K
FPS = 30
dt= 1./FPS


t=np.linspace(0,10,Nt)

yy0=2*L*np.random.random([N,1])-1*L
yy1=np.zeros([N,Nt-1])
yy=np.concatenate((yy0,yy1),axis=1)

xx0=2*L*np.random.random([N,1])-1*L
xx1=np.zeros([N,Nt-1])
xx=np.concatenate((xx0,xx1),axis=1)

vx0=.005*np.random.random([N,1])-.0025
vx1=np.zeros([N,Nt-1])
vx=np.concatenate((vx0,vx1),axis=1)

vy0=.005*np.random.random([N,1])-.0025
vy1=np.zeros([N,Nt-1])
vy=np.concatenate((vy0,vy1),axis=1)


s=np.linspace(0,1,10)
for k in range(1,L+1):
    sigma=0
    for n in range(1,Nt):
        for i in range(N):
            vx[i,n]=vx[i,n-1]  
            vy[i,n]=vy[i,n-1]
            xx[i,n]=xx[i,n-1]+vx[i,n-1]*t[n]
            yy[i,n]=yy[i,n-1]+vy[i,n-1]*t[n]

            #introducimos las condiciones de contorno FIJAS
            if abs(xx[i,n])>=k:
                vx[i,n]=-vx[i,n-1]
                sigma+=1
            if abs(yy[i,n])>=k:
                vy[i,n]=-vy[i,n-1]    #al chocar con las paredes, se aceleran!!
                sigma+=1
            #ahora introducimos las colisiones:
            for j in range(i,N):
                if (np.sqrt((xx[i,n]-xx[j,n])**2 +(yy[i,n]-yy[j,n])**2)<=R) & (i!=j):
                    Vcmx=0.5*(vx[i,n]+vx[j,n])
                    Vcmy=0.5*(vy[i,n]+vy[j,n])
                    modsep=np.sqrt((xx[i,n]-xx[j,n])**2+(yy[i,n]-yy[j,n])**2)
                    modv=np.sqrt((vx[i,n])**2+(vy[i,n])**2)

                    vx[i,n]=vx[i,n]-Vcmx-2*modv*(xx[i,n]-xx[j,n])/modsep
                    vy[i,n]=vy[i,n]-Vcmy-2*modv*(yy[i,n]-yy[j,n])/modsep
                    vx[j,n]=-vx[i,n]
                    vy[j,n]=-vy[i,n]
    s[k-1]=sigma/4                
    print('Con L =',k,', y N =',N,', se producen sigma/4 =',0.1*sigma/4,'colisiones.')
print(s)

Vemos que las colisiones son inversamente proporcionales a las dimensiones del lado de la caja.
Podemos verlo mejor en un gráfico:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
L=np.linspace(1,10,10)
x=np.linspace(1,10)
plt.scatter(L**2,s/10)
plt.xlabel('Dimensión de un lado de la caja')
plt.ylabel('Colisiones por segundo')
plt.show()
plt.plot(x,1/x**2,'k')
plt.show()


Vemos que se asemeja a una a una curva del tipo  $y(x)=1/x^2$.

Como la presión se puede definir como el número de choques de partículas contra una superficie, se puede ver que sigue una relación parecida a la ecuación de los gases ideales en dos dimensiones: $pA=Nk_BT$, donde la presión es inversamente proporcional al área a una temperatura dada.

En nuestro sistema $A=L^2$, así que sigue la relación de los gases ideales $p\propto L^{-2}$.